<a href="https://colab.research.google.com/github/SLCFLAB/Data-Science-Python/blob/main/Day%209/9_2_1_categorical_feature_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 범주형 데이터 이진분류 경진대회 베이스라인 모델

- [범주형 데이터 이진분류 경진대회 링크](https://www.kaggle.com/c/cat-in-the-dat/)

In [1]:
import pandas as pd

# 데이터 경로
data_path = 'https://raw.githubusercontent.com/SLCFLAB/Data-Science-Python/main/Day%209/data/'


train = pd.read_csv(data_path + 'cat_train.csv', index_col='id')
test = pd.read_csv(data_path + 'cat_test.csv', index_col='id')
submission = pd.read_csv(data_path + 'cat_sample_submission.csv', index_col='id')

## 1. 피처 엔지니어링

### 데이터 합치기

In [2]:
all_data = pd.concat([train, test]) # 훈련 데이터와 테스트 데이터 합치기 
all_data = all_data.drop('target', axis=1) # 타깃값 제거
all_data

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,...,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,...,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,...,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,...,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,...,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,0.0,0.0,1.0,T,N,Red,Trapezoid,Axolotl,Costa Rica,Theremin,...,c1a4acfaf,45a68dd2e,2.0,Expert,Warm,n,R,dp,2.0,12.0
999996,0.0,0.0,0.0,F,N,Red,Polygon,Dog,Russia,Theremin,...,220190c9e,e48348d66,1.0,Contributor,NaN,n,X,US,1.0,5.0
999997,0.0,0.0,1.0,T,N,Red,Triangle,Hamster,NaN,Theremin,...,cc5495ab3,47021df0d,1.0,Grandmaster,Boiling Hot,m,P,TL,1.0,6.0


### 원-핫 인코딩

In [3]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder() # 원-핫 인코더 생성
all_data_encoded = encoder.fit_transform(all_data) # 원-핫 인코딩 적용

### 데이터 나누기

In [4]:
num_train = len(train) # 훈련 데이터 개수

# 훈련 데이터와 테스트 데이터 나누기
X_train = all_data_encoded[:num_train] # 0 ~ num_train - 1행
X_test = all_data_encoded[num_train:] # num_train ~ 마지막 행

y = train['target']

In [5]:
from sklearn.model_selection import train_test_split

# 훈련 데이터, 검증 데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y,
                                                      test_size=0.1,
                                                      stratify=y,
                                                      random_state=10)

## 2. 모델 훈련

In [6]:
from sklearn.linear_model import LogisticRegression

logistic_model = LogisticRegression(max_iter=1000, random_state=42) # 모델 생성
logistic_model.fit(X_train, y_train) # 모델 훈련

LogisticRegression(max_iter=1000, random_state=42)

## 3. 모델 성능 검증

In [7]:
logistic_model.predict_proba(X_valid)

array([[0.77272251, 0.22727749],
       [0.91651808, 0.08348192],
       [0.75030512, 0.24969488],
       ...,
       [0.66952461, 0.33047539],
       [0.79398475, 0.20601525],
       [0.68582777, 0.31417223]])

In [8]:
logistic_model.predict(X_valid)

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
# 검증 데이터를 활용한 타깃 예측 
y_valid_preds = logistic_model.predict_proba(X_valid)[:, 1]

In [10]:
from sklearn.metrics import roc_auc_score # ROC AUC 점수 계산 함수

# 검증 데이터 ROC AUC
roc_auc = roc_auc_score(y_valid, y_valid_preds)

print(f'검증 데이터 ROC AUC : {roc_auc:.4f}')

검증 데이터 ROC AUC : 0.7858


## 4. 예측 및 결과 제출

In [11]:
# 타깃값 1일 확률 예측
y_preds = logistic_model.predict_proba(X_test)[:, 1]

In [12]:
# 제출 파일 생성
submission['target'] = y_preds
submission.to_csv('submission.csv')